<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/crawler/Finance_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    page = str(page)
    url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + page
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    div = bs_obj.find_all('div', {'class':'hotNewsList'})
    return [html.unescape(x['href']) for x in div[0].find_all('a')]

  def get_main(self, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/mainnews.nhn'
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return [x.find_all('a')[0]['href'] for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
    
  def get_breaking(self):
    pass

  # 시황 전망: 401
  # 기업 종목분석: 402
  # 해외 증시: 403
  # 채권 선물: 404
  # 공시 메모: 406
  # 환율: 429
  def get_focus(self, code=403, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3='\
     + str(code) +  '&date=' + str(date) + '&page=' + str(page)
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    temp = [html.unescape(x.find_all('a')[0]['href']) for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
    # 업데이트 가능
    return [x.replace('§', '&sect') for x in temp]

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    return title, subheading, text

In [2]:
news = News()

In [3]:
dic = {'Title':[], 'Subheading':[], 'Text':[]}

for i in news.get_ranking():
  title, subheading, text = news.get_article(i)
  dic['Title'].append(title)
  dic['Subheading'].append(subheading)
  dic['Text'].append(text)

In [4]:
pd.DataFrame.from_dict(dic)

,Title,Subheading,Text
0,"십만전자, 천슬라, 1억비트는 꿈이런가… 무너지는 개미들",선호 ‘3대장 종목’ 부진 늪 허덕… 금리 인상·테이퍼링 악재 겹쳐,선호 ‘3대장 종목’ 부진 늪 허덕… 금리 인상·테이퍼링 악재 겹쳐“삼성전자 10만...
1,인플레 우려에도 3200선…덜 오른 종목 찾기 `분주`,SK이노 호텔신라 실리콘웍스 등 2Q영업익 100%↑올해중 인플레 우려 지속…경기민...,SK이노 호텔신라 실리콘웍스 등 2Q영업익 100%↑올해중 인플레 우려 지속…경기민...
2,"계속되는 인플레 압박…""수출대형주 주목""",,[머니투데이 조준영 기자] [[개장전]](서울=뉴스1) 민경석 기자 = 31일 오후...
3,횡보장에 자사주 사들이는 기업들,지난달 관련 공시 16건시총 낮은 기업들도 적극 추진,지난달 관련 공시 16건시총 낮은 기업들도 적극 추진[서울=뉴시스]신항섭 기자 = ...
4,"""올해 코스피 3700 간다""…10개 증권사 센터장 증시 진단",,"[머니투데이 강민수 기자] [편집자주] 올초 코스피 3200, 코스닥 1000 돌파..."
5,"[특징주] 한화투자증권, 7조 '토스' 나스닥 IPO 기대감에 최대주주 지분 부각",,[헤럴드경제=이호 기자] 토스가 최근 투자 유치 과정에서 밸류에이션을 7조원 이상 ...
6,"[외환브리핑]中당국, 위안화 강세 완화 조치..환율, 1110원대 박스권 등락 전망","中 인민은행, 위안화 강세 랠리 제동뉴욕증시, 메모리얼데이로 휴장미 국채 금리 1....","中 인민은행, 위안화 강세 랠리 제동뉴욕증시, 메모리얼데이로 휴장미 국채 금리 1...."
7,오늘부터 양도세 인상 '최고 75%'…김오수 검찰총장 취임 [모닝브리핑],◆ 뉴욕증시 휴장…비트코인 2%·이더리움은 7% 올라,◆ 뉴욕증시 휴장…비트코인 2%·이더리움은 7% 올라뉴욕증시는 미국의 현충일 격인 ...
8,"[특징주]대신정보통신, 토스뱅크 IT센터 구축…제3 인터넷전문은행 출범 '실탄' 마련",,[아시아경제 박형수 기자] 대신정보통신이 강세다. 제3인터넷전문은행 토스뱅크의 출범...
9,[Asia오전]미국 휴장 속 혼조…日닛케이 하락,,[머니투데이 윤세미 기자] 1일 아시아 주요 증시가 엇갈린 흐름을 보이고 있다. 일...
